# building cross-compiler GNU toolchain

Cross-compiler build is going using GNU [make](make.ipynb) tool, which is the minimal and still powerful automation tool for build control.

## target system configuration

configuration variables can be overriden in a command line:
```
cd msp430 ; make APP=cnc HW=pc104 TARGET=i686 binutils
```

### application

In [106]:
%%writefile cross/Makefile

APP    ?= blink

Overwriting cross/Makefile


### hardware device

Hardware device you are targetting, must be used for lot of configuration: GPIO ports, peripherial pingout,..

* `HW`
  * [msp430g2]()
  * [cm0pill]()
  * [cm1pill]()

In [107]:
%%writefile -a cross/Makefile

HW     ?= msp430g2

Appending to cross/Makefile


### target MCU/CPU/OS triplet

* `BUILD`
  - system name of your workstation OS in form of **system triplet**
    - `x86_64-pc-linux-gnu`
* `HOST`
  - triplet of OS on which package will run
    * for MCU targets we'll drop it using `HOST=BUILD`
* `TARGET`
  - triplet of MCU target
    * [msp430](msp430.ipynb) MSP430
    * [arm-none-eabi](arm-none-eabi.ipynb) Cortex-M
    * [i486-elf](i486-elf.ipynb) PC104 VortexSX/DX

In [108]:
%%writefile -a cross/Makefile

-include  hw/$(HW)
-include cpu/$(CPU)

Appending to cross/Makefile


In [109]:
%%writefile cross/hw/msp430g2

CPU    ?= msp430f2553

Overwriting cross/hw/msp430g2


In [110]:
%%writefile cross/cpu/msp430f2553

TARGET ?= msp430

Overwriting cross/cpu/msp430f2553


## directory structure

* `Makefile`
* `hw/` hardware boards CPU & peripherials and specific tools defines
* `cpu/` CPU/MCU definitions

This set of directories will be created 

* `CWD` in the same dir as `Makefile` itself or directory selected by `make -C` option
* `GZ` downloaded archives
* `TMP` temp build directory
* `SRC` source code for every tool will be built
* `GNU` all compilers and tools will be pushed locally into this directory
  - such a scheme will not affect to whole `BUILD`/`HOST` system, all toolds will reside in a user `HOME`

In [111]:
%%writefile -a cross/Makefile

CWD = $(CURDIR)
GZ  = $(HOME)/gz
TMP = $(CWD)/tmp
SRC = $(TMP)/src
GNU = $(CWD)/$(TARGET)

Appending to cross/Makefile


In [112]:
%%writefile -a cross/Makefile

all: dirs gz binutils gdb

Appending to cross/Makefile


In [113]:
%%writefile -a cross/Makefile

dirs:
	mkdir -p $(GZ) $(TMP) $(SRC) $(GNU)

Appending to cross/Makefile


In [114]:
%%writefile cross/.gitignore
*~
*.swp
tmp/
msp430/

Overwriting cross/.gitignore


In [115]:
%%sh
cd cross ; make dirs ; ls -la ; ls -la tmp

mkdir -p /home/ponyatov/gz /home/ponyatov/jupy/cross/tmp /home/ponyatov/jupy/cross/tmp/src /home/ponyatov/jupy/cross/msp430
итого 36
drwxr-xr-x  7 ponyatov ponyatov 4096 май 29 21:27 .
drwxr-xr-x 12 ponyatov ponyatov 4096 май 29 21:36 ..
drwxr-xr-x  2 ponyatov ponyatov 4096 май 29 21:12 cpu
-rw-r--r--  1 ponyatov ponyatov   22 май 29 21:36 .gitignore
drwxr-xr-x  3 ponyatov ponyatov 4096 май 29 21:12 hw
drwxr-xr-x  2 ponyatov ponyatov 4096 май 29 21:09 .ipynb_checkpoints
-rw-r--r--  1 ponyatov ponyatov  239 май 29 21:36 Makefile
drwxr-xr-x  2 ponyatov ponyatov 4096 май 29 21:23 msp430
drwxr-xr-x  3 ponyatov ponyatov 4096 май 29 21:23 tmp
итого 12
drwxr-xr-x 3 ponyatov ponyatov 4096 май 29 21:23 .
drwxr-xr-x 7 ponyatov ponyatov 4096 май 29 21:27 ..
drwxr-xr-x 2 ponyatov ponyatov 4096 май 29 21:23 src


## cross-compiler components

## versions

In [116]:
%%writefile -a cross/Makefile

BINUTILS_VER = 2.32
GDB_VER      = 8.3

Appending to cross/Makefile


In [117]:
%%writefile -a cross/Makefile

BINUTILS = binutils-$(BINUTILS_VER)
GDB      = gdb-$(GDB_VER)

Appending to cross/Makefile


## predownload all archives

In [118]:
%%writefile -a cross/Makefile

BINUTILS_GZ = $(BINUTILS).tar.xz
GDB_GZ      = $(GDB).tar.xz

Appending to cross/Makefile


In [119]:
%%writefile -a cross/Makefile

WGET = wget -c -P $(GZ) -q

gz: $(GZ)/$(BINUTILS_GZ) $(GZ)/$(GDB_GZ)

Appending to cross/Makefile


In [127]:
%%sh
cd cross ; make gz ; ls -la ~/gz

wget -c -P /home/ponyatov/gz -q https://mirror.tochlab.net/pub/gnu/binutils/binutils-2.32.tar.xz
wget -c -P /home/ponyatov/gz -q http://ftp.gnu.org/gnu/gdb/gdb-8.3.tar.xz
итого 40316
drwxr-xr-x   2 ponyatov ponyatov     4096 май 29 21:37 .
drwxr-xr-x 181 ponyatov ponyatov    12288 май 29 21:13 ..
-rw-r--r--   1 ponyatov ponyatov 20774880 фев  2 20:38 binutils-2.32.tar.xz
-rw-r--r--   1 ponyatov ponyatov 20490300 май 11 22:47 gdb-8.3.tar.xz
-rw-r--r--   1 ponyatov ponyatov        0 окт 13  2017 .gitkeep


### archive unpack rules

In [121]:
%%writefile -a cross/Makefile

$(SRC)/%/README: $(GZ)/%.tar.xz
	cd $(SRC) ; xzcat $< | tar -x && touch $@

Appending to cross/Makefile


### configure scripts run

In [122]:
%%writefile -a cross/Makefile

XPATH = PATH=$(GNU)/bin:$(PATH)

CFG = configure --disable-nls --prefix=$(GNU)

MAKE = $(XPATH) make

Appending to cross/Makefile


most components can be built with use of full power of your computer, but `gcc` build can't be done in parallel even on i7/6G so `make gcc0` **will be run in single process** -- run in in background, and do some useful work while building: read docs, take a rest with tee, etc

In [123]:
%%writefile -a cross/Makefile

CPU_CORES = $(shell grep processor /proc/cpuinfo|wc -l)

MAKEJ = $(MAKE) -j$(CPU_CORES)

Appending to cross/Makefile


### [binutils](binutils.ipynb) : assembler, linker, object file tools

In [124]:
%%writefile -a cross/Makefile

binutils: $(GNU)/bin/$(TARGET)-ld

CFG_BINUTILS += --target=$(TARGET)

$(GNU)/bin/$(TARGET)-ld: $(GZ)/$(BINUTILS_GZ) $(SRC)/$(BINUTILS)/README
	rm -rf $(TMP)/$(BINUTILS) ; mkdir $(TMP)/$(BINUTILS) ; cd $(TMP)/$(BINUTILS) ;\
	$(XPATH) $(SRC)/$(BINUTILS)/$(CFG) $(CFG_BINUTILS) && $(MAKEJ) && $(MAKE) install-strip

$(GZ)/$(BINUTILS_GZ):
	$(WGET) https://mirror.tochlab.net/pub/gnu/binutils/$(BINUTILS_GZ)

Appending to cross/Makefile


In [125]:
%%sh
# cd msp430 ; make binutils ; ls -la tmp/src tmp gnu*

after `make` run `configure` will be run with required options:

`/home/dpon/jupy/msp430/tmp/src/binutils-2.32/configure --disable-nls --prefix=/home/user/jupy/msp430 --target=msp430`

detecting host/target

```
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking target system type... msp430-unknown-none
```

build requires some preinstalled developmen packages:
 * `sudo apt install`
   - `build-essential` `m4` `wget` `make` `gcc g++` `flex` `bison`
   - `xz` `bzip2`
   - `bc` `libncurses-dev` only for Linux kernel build

```
checking for gcc... gcc
checking for g++... g++
checking for bison... bison
checking for m4... m4
checking for flex... flex
```

`make -j4`

`make install-strip`

### [gdb](gdb.ipynb) : remote hardware debugger

In [126]:
%%writefile -a cross/Makefile

gdb: $(GNU)/bin/$(TARGET)-gdb

CFG_GDB += $(CFG_BINUTILS) --enable-python

$(GNU)/bin/$(TARGET)-gdb: $(GZ)/$(GDB_GZ) $(SRC)/$(GDB)/README
	rm -rf $(TMP)/$(GDB) ; mkdir $(TMP)/$(GDB) ; cd $(TMP)/$(GDB) ;\
	$(XPATH) $(SRC)/$(GDB)/$(CFG) $(CFG_GDB) && $(MAKEJ) && $(MAKE) install-strip

$(GZ)/$(GDB_GZ):
	$(WGET) http://ftp.gnu.org/gnu/gdb/$(GDB_GZ)

Appending to cross/Makefile
